In [1]:
# Cell 1: Load the OpenSubtitles English-French Dataset
from datasets import load_dataset
import torch
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
# Load the dataset
dataset = load_dataset("opus100", "en-fr")

# Inspect the dataset
print(dataset)


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

c:\Users\srinivas\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\srinivas\.cache\huggingface\hub\datasets--opus100. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/334k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Cell 2: Preprocess the Data
from transformers import MarianTokenizer

# Load the tokenizer
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

# Define source and target languages
SRC_LANG = "en"
TGT_LANG = "fr"

# Preprocessing function
def preprocess_function(examples):
    inputs = [ex[SRC_LANG] for ex in examples['translation']]
    targets = [ex[TGT_LANG] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the preprocessing to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)


c:\Users\srinivas\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

c:\Users\srinivas\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4114: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [4]:
# Cell 3: Set Up Evaluation Metrics
import evaluate

# Load BLEU metric
bleu = evaluate.load('sacrebleu')

# Metric computation function
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = [[tokenizer.decode(l, skip_special_tokens=True)] for l in labels]
    result = bleu.compute(predictions=decoded_preds, references=labels)
    return {"bleu": result["score"]}


In [9]:
del model

In [5]:
# Cell 4: Load the Pre-trained MarianMT Model
from transformers import MarianMTModel

# Load the model
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')


In [8]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")


Total parameters: 75133952
Trainable parameters: 74609664


In [ ]:
# Cell 5: Evaluate the Base Model with GPU


# Check if GPU is available


# Split the data
split_datasets = tokenized_datasets['train'].train_test_split(test_size=0.1)
train_dataset = split_datasets['train']
eval_dataset = split_datasets['test']

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_eval_batch_size=24,
    predict_with_generate=True,
    dataloader_pin_memory=True,  # Ensures better performance with GPUs
)
small_train_dataset = train_dataset.shuffle(seed=42).select(range(30000))
small_eval_dataset = eval_dataset.shuffle(seed=42).select(range(1000))
# Use a data collator to handle padding during evaluation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model.to(device),  # Send the model to GPU
    args=training_args,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Evaluate the base model
base_metrics = trainer.evaluate()
print(f"Base Model BLEU Score: {base_metrics['eval_bleu']:.2f}")


In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()


In [ ]:
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-en-fr')

In [ ]:
# Cell 6: Fine-tune the Model with Proper Padding
from transformers import DataCollatorForSeq2Seq

# Update training arguments for training
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-9,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=2,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    dataloader_pin_memory=True,  # Helps with GPU training efficiency
)

# Use a data collator to handle padding during training and evaluation
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Re-initialize the trainer with training dataset and data collator
trainer = Seq2SeqTrainer(
    model=model.to("cuda"),  # Move model to GPU
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=small_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Start fine-tuning
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-9a2678ddf558>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[59513]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Bleu
1,1.488300,1.343655,38.690481
2,1.487300,1.341634,38.738641


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

TrainOutput(global_step=14064, training_loss=1.488615716819199, metrics={'train_runtime': 7013.6143, 'train_samples_per_second': 256.644, 'train_steps_per_second': 2.005, 'total_flos': 5.09732086307881e+16, 'train_loss': 1.488615716819199, 'epoch': 2.0})